## Observations and Insights 

In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
merge_df = pd.merge(mouse_metadata, study_results, on="Mouse ID")
merge_df.rename(columns={'Age_months':'Age (Months)', 'Weight (g)':'Weight (Grams)', 'Timepoint':'Time (Days)'}, inplace=True)
# Display the data table for preview
merge_df

,Mouse ID,Drug Regimen,Sex,Age (Months),Weight (Grams),Time (Days),Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [2]:
# Checking the number of mice.
total_mice = merge_df['Mouse ID'].nunique()
print(f'Total Mice: {total_mice}')

Total Mice: 249


In [3]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicate_mice = merge_df[merge_df.duplicated(keep=False)]
duplicate_mice

,Mouse ID,Drug Regimen,Sex,Age (Months),Weight (Grams),Time (Days),Tumor Volume (mm3),Metastatic Sites
908,g989,Propriva,Female,21,26,0,45.0,0
909,g989,Propriva,Female,21,26,0,45.0,0


In [4]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_mice_df = merge_df.drop(merge_df[merge_df['Mouse ID'] == 'g989'].index)

clean_mice_df.head(912)

,Mouse ID,Drug Regimen,Sex,Age (Months),Weight (Grams),Time (Days),Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
907,g867,Stelasyn,Female,9,25,15,51.060419,1
921,h246,Ketapril,Male,13,30,0,45.000000,0
922,h246,Ketapril,Male,13,30,5,49.470417,0
923,h246,Ketapril,Male,13,30,10,51.368862,1


In [5]:
# Checking the number of mice in the clean DataFrame.
clean_mice_df.nunique()

Mouse ID               248
Drug Regimen            10
Sex                      2
Age (Months)            24
Weight (Grams)          16
Time (Days)             10
Tumor Volume (mm3)    1633
Metastatic Sites         5
dtype: int64

## Summary Statistics

In [38]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

tumor_vol_mean = clean_mice_df.groupby(['Drug Regimen'])['Tumor Volume (mm3)'].mean()
tumor_vol_median = clean_mice_df.groupby(['Drug Regimen'])['Tumor Volume (mm3)'].median()
tumor_vol_var = clean_mice_df.groupby(['Drug Regimen'])['Tumor Volume (mm3)'].var()
tumor_vol_sd = clean_mice_df.groupby(['Drug Regimen'])['Tumor Volume (mm3)'].std()
tumor_vol_sem = clean_mice_df.groupby(['Drug Regimen'])['Tumor Volume (mm3)'].sem()

tumor_vol_statistics_df = pd.DataFrame({'TVOL Mean': tumor_vol_mean, 
                                     'TVOL Median': tumor_vol_median, 
                                     'TVOL Variance': tumor_vol_var,
                                     'TVOL Std Dev': tumor_vol_sd,
                                     'TVOL SEM': tumor_vol_sem})
tumor_vol_statistics_df

,TVOL Mean,TVOL Median,TVOL Variance,TVOL Std Dev,TVOL SEM
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [57]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
agg_stats = clean_mice_df.loc[:, ['Drug Regimen', 'Tumor Volume (mm3)']]
agg_summary_df.groupby('Drug Regimen').agg([np.mean ,np.median, np.var, np.std])
# Using the aggregation method, produce the same summary statistics in a single line


AttributeError: 'SeriesGroupBy' object has no attribute 'groupby'

In [58]:
agg_stats

,Drug Regimen,Tumor Volume (mm3)
0,Ramicane,45.000000
1,Ramicane,38.825898
2,Ramicane,35.014271
3,Ramicane,34.223992
4,Ramicane,32.997729
...,...,...
1888,Naftisol,63.145652
1889,Naftisol,65.841013
1890,Naftisol,69.176246
1891,Naftisol,70.314904


## Bar and Pie Charts

In [28]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.



In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
